In [15]:
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import csv
import time
import re
import pandas as pd
import requests
import sys

In [2]:
# set driver options - do not pop up chrome and disable pop-ups.
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument(f'user-agent":{UserAgent().random}')

# get appropriate drivers for Google Chrome - automatic without downloading driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                          options=chrome_options)

In [136]:
pattern = re.compile(r'post-author thb-post-author-\d+')
# url to scrape 
url = 'https://www.therakyatpost.com/?s=vaksin'
driver.get(url)

# Scroll to the bottom of the page and wait for a few seconds for the page to load
while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for the page to load
    time.sleep(3)

    # Check if we have reached the bottom of the page
    scroll_position = driver.execute_script("return window.pageYOffset;")
    page_height = driver.execute_script("return document.body.scrollHeight;")

    # The position for the end of page, if it is reached, quit browser
    # page_height > scroll_height does not work 
    
    if ((scroll_position == 86940) & (page_height == 87540)):
        break

page_source = driver.page_source
# Close the browser  
driver.quit()

soup = BeautifulSoup(page_source, 'html.parser')

links = soup.find_all('div', attrs={'class':'post-title'})
print(f'Links we obtained: {len(links)}')

link_date = soup.find_all('li', attrs={'class':'post-date'})
print(f'link_with_date we obtained: {len(link_date)}')

link_author = soup.find_all('div', attrs={'class': pattern})
print(f'link_with_author we obtained: {len(link_author)}')


7539.5
8139
7539.5
15576
14975.5
15576
14975.5
15576
14975.5
15576
14975.5
23124
22524
23124
22524
23124
22524
30700
30100
30700
30100
30700
30100
37982
37382.5
37982
37382.5
37982
37382.5
45448
44848
45448
44848
52772
52172
52772
52172
52772
52172
52772
52172
52772
52172
60264
59664.5
60264
59664.5
60264
59664.5
60264
59664.5
60264
59664.5
67463
66862.5
67463
66862.5
67463
66862.5
67463
66862.5
74801
74201
74801
74201
74801
74201
82531
81931.5
82531
81931.5
82531
81931.5
87540
86940
87540
Links we obtained: 349
link_with_date we obtained: 348
link_with_author we obtained: 348


In [139]:
# Writing to a file 
with open('the_rakya_post.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    headers = ["Article_Title", "Date", "Link", "Author Name"]
    writer.writerow(headers)

    for link, date, author in zip(links, link_date, link_author):
        # in div tag, go in and search for 'a' tags
        # Ex: Warga Emas 60 Tahun & Ke Atas Digesa Ambil Suntikan Vaksin Influenza
        article_title = link.find('a').text

        # Ex: September 8, 2022 
        date = date.text 

        # Ex: 2022-09-08 00:00:00 --> 2022-09-08
        dateobj = datetime.strptime(date, '%B %d, %Y').date()

        # in a's tag, get its link
        Link = link.find('a')['href']

        author_name = author.find('a').text
        
        writer.writerow([article_title, dateobj, Link, author_name])

    print('CSV file saved successfully')


CSV file saved successfully


In [19]:
# read csv files and drop duplicates

site_data = pd.read_csv('the_rakya_post.csv')
# site_data = site_data.drop_duplicates()
site_data

,Article_Title,Date,Link,Author Name
0,Warga Emas 60 Tahun & Ke Atas Digesa Ambil Sun...,2022-09-08,https://www.therakyatpost.com/trpbm/berita/202...,Siti Murni
1,Lelaki Ini Kongsi Prosedur Melancong Ke Thaila...,2022-05-17,https://www.therakyatpost.com/trpbm/jalan-jala...,Farahin Fadzil
2,15 Mei Sebagai Tarikh Akhir Pemberian Dos Vaks...,2022-04-14,https://www.therakyatpost.com/trpbm/berita/202...,Farahin Fadzil
3,Kesatuan Eropah Iktiraf Sijil Vaksinasi Digita...,2022-04-05,https://www.therakyatpost.com/trpbm/berita/202...,Farahin Fadzil
4,Status Vaksin Sinovac Cuma Bertukar Tidak Leng...,2022-03-29,https://www.therakyatpost.com/trpbm/viral/2022...,Farahin Fadzil
...,...,...,...,...
343,Will You Take The Covid-19 Vaccine? MOH Wants ...,2020-12-22,https://www.therakyatpost.com/news/malaysia/20...,Tasneem Nazari
344,Covid-19: PM Muhyiddin Among First To Be Vacci...,2020-12-06,https://www.therakyatpost.com/news/malaysia/20...,Tasneem Nazari
345,Covid-19 Vaccine: Health D-G Responds To Reute...,2020-12-01,https://www.therakyatpost.com/news/malaysia/20...,Anne Dorall
346,Pneumococcal Vaccine Now FREE In Klinik Kesiha...,2020-11-30,https://www.therakyatpost.com/news/malaysia/20...,Tasneem Nazari


In [21]:
# find p tags and combine into one article 
article_text = []
for url, link_index in zip(site_data['Link'], range(len(site_data['Link']))):
    print('Processing link: ',link_index)
    
    try:
      # response is equivalent to enter a key in chrome 
      # prevent ip-block by adding fake devices accessing web pages 
      response = requests.get(url, headers={'User-Agent': UserAgent().random})

      # this link give valid status code: 200 --> web scrap pass 
      # print(page.status_code)

    except Exception as e:
      error_type, error_obj, error_info = sys.exc_info()
      print('Error Link: ', url)
      print(error_type, 'Line: ', error_info.tb_lineno)

      # ignore this paage and move on to next one
      continue 

    # delay by 2 seconds to prevent ip block
    time.sleep(2)

    soup_article = BeautifulSoup(response.text, 'html.parser')
    post_content = soup_article.find_all('div', attrs={'class':'post-content entry-content'})

    article_paragraphs = soup_article.find_all('p')
    combined_article = ''
    
    for p in article_paragraphs:
        combined_article += p.get_text()
    article_text.append(combined_article)
    print('Obtained article text for link: ',link_index)

Processing link:  0
Obtained article text for link:  0
Processing link:  1
Obtained article text for link:  1
Processing link:  2
Obtained article text for link:  2
Processing link:  3
Obtained article text for link:  3
Processing link:  4
Obtained article text for link:  4
Processing link:  5
Obtained article text for link:  5
Processing link:  6
Obtained article text for link:  6
Processing link:  7
Obtained article text for link:  7
Processing link:  8
Obtained article text for link:  8
Processing link:  9
Obtained article text for link:  9
Processing link:  10
Obtained article text for link:  10
Processing link:  11
Obtained article text for link:  11
Processing link:  12
Obtained article text for link:  12
Processing link:  13
Obtained article text for link:  13
Processing link:  14
Obtained article text for link:  14
Processing link:  15
Obtained article text for link:  15
Processing link:  16
Obtained article text for link:  16
Processing link:  17
Obtained article text for link

In [22]:
site_data['Content'] = article_text
site_data.to_csv('rakya_post_with_text.csv', encoding='utf-8', index=False)
site_data

,Article_Title,Date,Link,Author Name,Content
0,Warga Emas 60 Tahun & Ke Atas Digesa Ambil Sun...,2022-09-08,https://www.therakyatpost.com/trpbm/berita/202...,Siti Murni,Selesema merupakan penyakit respiratori yang b...
1,Lelaki Ini Kongsi Prosedur Melancong Ke Thaila...,2022-05-17,https://www.therakyatpost.com/trpbm/jalan-jala...,Farahin Fadzil,Penerima vaksin jenis Sinovac perlu lengkap ti...
2,15 Mei Sebagai Tarikh Akhir Pemberian Dos Vaks...,2022-04-14,https://www.therakyatpost.com/trpbm/berita/202...,Farahin Fadzil,Pendaftaran vaksin Covid-19 untuk golongan kan...
3,Kesatuan Eropah Iktiraf Sijil Vaksinasi Digita...,2022-04-05,https://www.therakyatpost.com/trpbm/berita/202...,Farahin Fadzil,Sijil vaksinasi digital di dalam aplikasi MySe...
4,Status Vaksin Sinovac Cuma Bertukar Tidak Leng...,2022-03-29,https://www.therakyatpost.com/trpbm/viral/2022...,Farahin Fadzil,Pihak Kementerian Kesihatan Malaysia (KKM) men...
...,...,...,...,...,...
343,Will You Take The Covid-19 Vaccine? MOH Wants ...,2020-12-22,https://www.therakyatpost.com/news/malaysia/20...,Tasneem Nazari,They want to see if the masses would accept or...
344,Covid-19: PM Muhyiddin Among First To Be Vacci...,2020-12-06,https://www.therakyatpost.com/news/malaysia/20...,Tasneem Nazari,The government has struck another vaccine deal...
345,Covid-19 Vaccine: Health D-G Responds To Reute...,2020-12-01,https://www.therakyatpost.com/news/malaysia/20...,Anne Dorall,Will we still be getting our vaccine? Probably...
346,Pneumococcal Vaccine Now FREE In Klinik Kesiha...,2020-11-30,https://www.therakyatpost.com/news/malaysia/20...,Tasneem Nazari,"If there’s a way to protect our children, why ..."
